http://rail.eecs.berkeley.edu/deeprlcourse/static/homeworks/hw1.pdf

In [1]:
import pickle
import keras

Using TensorFlow backend.


In [2]:
roll_out_file = 'expert_data/RoboschoolHalfCheetah-v1.pkl'

In [3]:
# read roll-outs
with open(roll_out_file, 'rb') as f:
    roll_outs = pickle.loads(f.read())

In [4]:
print(roll_outs['observations'].shape)
print(roll_outs['actions'].shape)

(1000000, 26)
(1000000, 6)


In [5]:
roll_outs['actions'][0]

array([ 1.334986  ,  0.182585  , -0.7752828 ,  1.2846712 ,  0.49695894,
        1.6057699 ], dtype=float32)

In [6]:
from keras.layers import Input, Dense
from keras.models import Model

def create_model(input_size, output_size):
    inputs = Input(shape=(input_size,))
    
    x = Dense(30, activation='relu')(inputs)
#     x = keras.layers.Dropout(0.1)(x)
#     x = Dense(50, activation='relu')(x)
#     x = keras.layers.Dropout(0.2)(x)
#     x = Dense(20, activation='relu')(x)
    x = Dense(10, activation='relu')(x)
    
    predictions = Dense(output_size)(x)
    
    return Model(inputs=inputs, outputs=predictions)

In [7]:
model = create_model(26, 6)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                810       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 66        
Total params: 1,186
Trainable params: 1,186
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
X = roll_outs['observations']
Y = roll_outs['actions']

In [11]:
# model.fit(X, Y, epochs=20, batch_size=100)

In [12]:
# model.save_weights("model")

In [13]:
model.load_weights("model")

In [15]:
# test our model with actual environment
import gym
import roboschool
env = gym.make('RoboschoolHalfCheetah-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [16]:
for _ in range(10):
    obs = env.reset()
    for _ in range(1100):
        env.render("human")
        obs, reward, done, _ = env.step(model.predict(obs.reshape((1,-1))).reshape((-1)))


In [17]:
env.close()